In [ ]:
# Load dataset and model if not continuing from before
import torch
import torchvision

from datasets import load_from_disk
from tqdm import tqdm
import datetime
from pathlib import Path
import time


dataset_path = "../datasets/byol/grasp_100_2024-09-06_17-03-47.hf"
dataset_name = Path(dataset_path).stem
dataset = load_from_disk(dataset_path)
dataset.set_format("torch")

checkpoint = torch.load("../VINN/ckpts/resnet_byol_grasp_100_2024-09-06_17-03-47_2024-10-12_21-08/epoch_10.pt")
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("mps")

# Remove the fc/classification layer
resnet = torchvision.models.resnet18()
modules = list(resnet.children())[:-1]
backbone = torch.nn.Sequential(*modules)
net = backbone
net.load_state_dict(checkpoint["policy_state_dict"])
net = net.to(device)


In [ ]:
import gym_lite6.env, gym_lite6.pickup_task, gym_lite6.utils
import gymnasium as gym
import numpy as np
import mediapy as media


task = gym_lite6.pickup_task.GraspAndLiftTask('gripper_left_finger', 'gripper_right_finger', 'box', 'floor')
env = gym.make(
    "UfactoryCubePickup-v0",
    task=task,
    obs_type="pixels_state",
    max_episode_steps=300,
)
observation, info = env.reset()
media.show_image(env.render(), width=400)
